#### 讀取模型資料

In [163]:
import OCC.Extend.DataExchange
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file


#讀取模型
iges_file =  r"C:\alan\ITRI\model\simple_1.IGS"
model = read_iges_file(iges_file)
# Get the top level shape
# model = iges_reader.OneShape()

# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(model, update=True)
start_display()

#### 計算每殼層面積

In [164]:
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopoDS import TopoDS_Shape

def calculate_surface_area(shape):
    # 將 shape 轉換為 TopoDS_Shape
    topods_shape = TopoDS_Shape(shape)

    # 初始化表面屬性
    surface_props = GProp_GProps()

    # 計算表面屬性
    brepgprop_SurfaceProperties(topods_shape, surface_props)
    

    # 獲取表面積
    surface_area = surface_props.Mass()

    return surface_area

#### 輸入起始點、終點座標、法向量，生成掃除路徑

In [165]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir,gp_Circ, gp_Ax2, gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Display.SimpleGui import init_display
import numpy as np
# 輸入: 兩個點
# 輸出: 線段
# def generate_segment(point1, point2):
#     aSegment = GC_MakeSegment(gp_Pnt(*point1), gp_Pnt(*point2))
#     anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
#     aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())
#     return aWire.Shape()

def grinding_area(point_start,point_end,normal_vector):
    #根據起始、終點生成線段
    point_s=gp_Pnt(float(point_start[0]),float(point_start[1]), float(point_start[2]))
    point_e=gp_Pnt(float(point_end[0]),float(point_end[1]), float(point_end[2]))
    aSegment = GC_MakeSegment(point_s, point_e )
    anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
    aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())

    #input
    #normal_vector
    magnitude = np.linalg.norm(normal_vector)# 將法向量除以其範數以得到單位法向量
    normal_vector = normal_vector/ magnitude

    #two point_position
    tan_vector=point_start-point_end

    #direction_1
    dir_1= np.cross(normal_vector, tan_vector)
    magnitude = np.linalg.norm(dir_1)# 將向量除以其範數以得到單位向量
    dir_1 = dir_1/ magnitude

    # 定義截面為方形
    # 定義方形的四個頂點
    point_1=point_start+normal_vector*0.5+dir_1*5
    point_2=point_start+normal_vector*0.5-dir_1*5
    point_3=point_start-normal_vector*0.5-dir_1*5
    point_4=point_start-normal_vector*0.5+dir_1*5
    p1 = gp_Pnt(point_1[0], point_1[1], point_1[2])
    p2 = gp_Pnt(point_2[0], point_2[1], point_2[2])
    p3 = gp_Pnt(point_3[0], point_3[1], point_3[2])
    p4 = gp_Pnt(point_4[0], point_4[1], point_4[2])

    # 使用這些頂點創建方形的邊緣
    edge1 = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
    edge2 = BRepBuilderAPI_MakeEdge(p2, p3).Edge()
    edge3 = BRepBuilderAPI_MakeEdge(p3, p4).Edge()
    edge4 = BRepBuilderAPI_MakeEdge(p4, p1).Edge()

    # 使用這些邊緣創建方形的線框
    square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3, edge4).Wire()

    # 將線框轉換為面，作為截面
    F = BRepBuilderAPI_MakeFace(square_wire, True)
    S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟

    return S

#### 等比例縮放

In [166]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.gp import gp_Trsf,gp_Pnt,gp_Vec
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Display.OCCViewer import rgb_color
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def zoom(model,m_center,mag):
    t1 = gp_Trsf()
    #物件質心位置,單位mm
    t1.SetScale(m_center, mag)

    # 对形状进行缩放
    scaled_box = BRepBuilderAPI_Transform(model, t1, True).Shape()


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # display.DisplayShape(scaled_box, update=True, color=rgb_color(0, 0, 0))
    # display.DisplayShape(model, update=True, color=rgb_color(1, 0, 0))
    # start_display()
    return scaled_box






#### 掃除

In [167]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader

def grind(model,S):
    # 创建一个几何切割对象
    cut_algo = BRepAlgoAPI_Cut(model,S.Shape() )
    # 执行几何切割操作
    cut_result = cut_algo.Shape()
    return cut_result


#### 對每一殼層進行掃除，並儲存掃除結果

In [216]:
from OCC.Extend.DataExchange import write_iges_file
import os
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
#讀取模型
iges_file =  r"C:\alan\ITRI\model\simple_1.IGS"
model = read_iges_file(iges_file)

point_start=np.array((-40,50,0))
point_end=np.array((40,50,0))
normal_vector=np.array((0,1,0))
S=grinding_area(point_start,point_end,normal_vector)
m_center= gp_Pnt(0, 0, 0)#質心位置
#simple_1:m_center=(0,0,0)
#simple_2:m_center=(0,-2.2,-2.2)
#simple_3:m_center=(-0.36,-0.76,-0.36)
#complex_3:m_center= gp_Pnt(0, 0, 61.5)

i=0
while i<3:
    mag=1-i*0.05
    Model=zoom(model,m_center,mag)
    cut_result=grind(Model,S)
    
    surface_origin = calculate_surface_area(Model)
    print(f"Model_{i}_origin_area:", surface_origin) 
    surface_cut_result=calculate_surface_area(cut_result)
    print(f"Model_{i}_final_area:", surface_cut_result)


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # box=display.DisplayShape(cut_result, update=True)[0]
    # box.SetTransparency(0.1)
    # start_display()
    if i==0:
        all_result=cut_result
        # all_result = all_result.Shape()
    else:
        all_result = BRepAlgoAPI_Fuse(all_result,cut_result)
        all_result = all_result.Shape()

    iges_file_path=os.path.join(r"C:\alan\ITRI\cut_result",f'layer_{i}.igs' )
    write_iges_file(cut_result, iges_file_path)
    i=i+1

iges_file_path=os.path.join(r"C:\alan\ITRI\cut_result\all.igs" )
write_iges_file(all_result, iges_file_path)

Model_0_origin_area: 60000.0
Model_0_final_area: 59200.0
Model_1_origin_area: 54150.0
Model_1_final_area: 54150.0
Model_2_origin_area: 48600.0
Model_2_final_area: 48600.0


C:\Users\Martin\AppData\Local\Temp\ipykernel_9940\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\Martin\AppData\Local\Temp\ipykernel_9940\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\Martin\AppData\Local\Temp\ipykernel_9940\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfacePrope

#### 檢視各層研磨結果

In [217]:
from OCC.Core.Quantity import Quantity_Color
#讀取模型
iges_file = r"C:\alan\ITRI\cut_result\all.igs"
cut_result = read_iges_file(iges_file)

display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(cut_result, update=True)[0]
box.SetTransparency(0.5)
start_display()


In [205]:
from OCC.Core.TopoDS import TopoDS_Compound
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
# 创建一个空的复合形状
all = BRepAlgoAPI_Fuse(cut_result,cut_result_1)
all = all.Shape()
iges_file_path=os.path.join(r"C:\alan\ITRI\cut_result\all.igs" )
write_iges_file(all, iges_file_path)


In [206]:
from OCC.Core.Quantity import Quantity_Color
#讀取模型
iges_file = r"C:\alan\ITRI\cut_result\all.igs"
result = read_iges_file(iges_file)

# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(result, update=True)[0]
box.SetTransparency(0.5)


start_display()